# Crack Detection In Buildings

Crack detection has vital importance for structural health monitoring and inspection. We would like to train a network to detect Cracks, we will denote the images that contain cracks as positive and images with no cracks as negative.

To train the network, we will use a pre-trained model to classify between the positive and negetive samples. The particular **pre-trained model** will be **resnet18**.

## Download the data

Download the dataset and unzip the files in the data directory. (This may take some time)

In [5]:
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/images/Positive_tensors.zip 

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
!unzip -q Positive_tensors.zip 

'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
! wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/images/Negative_tensors.zip
!unzip -q Negative_tensors.zip

'wget' is not recognized as an internal or external command,
operable program or batch file.
'unzip' is not recognized as an internal or external command,
operable program or batch file.


We will install torchvision:

In [4]:
!pip install torchvision

ERROR: Could not find a version that satisfies the requirement torch==1.4.0 (from torchvision) (from versions: 0.1.2, 0.1.2.post1, 0.1.2.post2)
ERROR: No matching distribution found for torch==1.4.0 (from torchvision)


## Imports and Auxiliary Functions

The following are the libraries we are going to use for this lab. The torch.manual_seed() is for forcing the random function to give the same number every time we try to recompile it.



In [ ]:
import torchvision.models as models
from PIL import Image
import pandas as pd
from torchvision import transforms
import torch.nn as nn
import torch 
import matplotlib.pylab as plt
import numpy as np
from torch.utils.data import Dataset, DataLoader
import os
torch.manual_seed(0)

In [ ]:
from matplotlib.pyplot import imshow
import time